# **Downwelling irradiance model**
## Auxiliary functions

This notebook contains auxiliary functions that are required in the notebook `irradiance_model.ipynb`.

**Author:** [Bruno Rech](https://github.com/b-rech) | **Created on:** 21 January 2025

* * *

### Dependencies

Read the documentation at the repository for further details.

In [1]:
# Import libraries
import os
import ee
import numpy as np
import pandas as pd
from scipy import stats

# Inicialize GEE
try:

    ee.Initialize()

except:

    ee.Authenticate()
    ee.Initialize()

### Function to open field data 

In [23]:
def openFieldData(filename, equipment, tz_input, tz_output):
    '''
    This function is used to open and configure irradiance files.

    ---
    filename: path, relative path or filename to be loaded.
    
    equipment: spectroradiometer used to acquire field data
               (either 'satlantic' or 'trios').
               
    tz_input: time zone of the datetimes in the input file
               (usually 'UTC' for Satlantic and
                'America/Sao_Paulo' for TriOS).
                
    tz_output: time zone of interest for the outputs.
    '''

    # Processing workflow for the Satlantic spectroradiometer
    if equipment == 'satlantic':
        
        # Open data
        field_data = pd.read_csv(filename).dropna()
        
        # Retrieve acquisition time
        times = [f'{x[:2]}:{x[2:4]}:{x[4:6]}' 
                 for x in field_data.time.astype(str)]
        
        # Configure time
        field_data['time'] = (
            
            pd.to_datetime(
                times,
                format='%H:%M:%S')
            .tz_localize(tz_input)
            .tz_convert(tz_output)

        )
        
        # Columns to keep
        cols = (['X' + str(wl) for wl in range(360, 801)] +
               ['time', 'station_id'])
        
        # Get spectra medians
        es = field_data[cols].groupby('station_id').median()
        
        # Save times as text
        es['time'] = [x.strftime('%H:%M') 
                      for x in es.time.dt.round('min')]
      
        # Set time as row index
        es.set_index('time', inplace=True)

        # Configure table shape and convert
        # from μW/cm²/nm to W/m²/nm
        es = es.transpose() / 100
        
        # Adjust indices
        es.rename_axis('wl', axis=0, inplace=True)
        es.reset_index(drop=False, inplace=True)
        es.rename_axis(None, axis=1, inplace=True)

        # Adjust wavelengths
        es['wl'] = [int(x[1:]) for x in es.wl]

        return es

    elif equipment == 'trios':

        # Open data
        field_data = pd.read_csv(filename)

        # Configure datetimes
        field_data['DateTime'] = (
            
            pd.to_datetime(field_data.DateTime,
                           format='%Y-%m-%d %H:%M:%S')
            .dt.tz_localize(tz_input)
            .dt.tz_convert(tz_output)

        )

        # Columns to keep
        cols = ([str(x) for x in range(400, 901)] + 
                ['estacoes_id', 'DateTime'])

        # Get median values
        es = field_data[cols].groupby('estacoes_id').median()

        # Create a column of time as text
        es['time'] = [x.strftime('%H:%M') 
                      for x in es.DateTime.dt.round('min')]

        # Drop datetime columns
        es.drop('DateTime', axis=1, inplace=True)

        # Set time as index
        es = es.set_index('time')

        # Change shape and convert
        # from mW/m²/nm to W/m²/nm
        es = es.transpose() / 1000

        # Adjust indices
        es.rename_axis('wl', axis=0, inplace=True)
        es.reset_index(drop=False, inplace=True)
        es.rename_axis(None, axis=1, inplace=True)

        # Adjust wavelengths
        es['wl'] = es.wl.astype(int)
        
        return es
    
    else:

        raise Exception('Indicate the equipment!')

### Function to get atmospheric parameters from GEE

In [ ]:
def atmParams(date, roi):
    '''
    This function retrieves atmospheric parameters from
    products available at Google Earth Engine.

    ---
    date: date of interest ('yyyy-mm-dd').

    roi: an ee.Feature delimitating the region of interest.
    '''

    # Configure date
    date = pd.to_datetime(date)

    # Iterate to find the shortest timedelta
    for td_aod in range(0, 100):

        try:

            # Get aerossol optical depth (dimensionless)
            aod_550 = (
                
                ee.ImageCollection('MODIS/061/MCD19A2_GRANULES')
                .filterDate(date - pd.Timedelta(td_aod, 'd'),
                            date + pd.Timedelta(td_aod + 1, 'd'))
                .filterBounds(roi)
                .select('Optical_Depth_055')
                .mean()
                .reduceRegion(reducer=ee.Reducer.mean(),
                              geometry=roi,
                              scale=1000)
                .getNumber('Optical_Depth_055')
                .multiply(0.001)
                .getInfo()

            )

            break
    
        except:
    
            continue
    
    print(f'Mean AOD: {aod_550:.3f} (time window: {2 * td_aod + 1})')
    
    # Ozone [atm-cm] 
    # The conversion factor from mol/m² to atm-cm is 2.238
    ozone = (
        
        ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_O3')
        .filterDate(date, date + pd.Timedelta(1, 'd'))
        .filterBounds(roi)
        .select('O3_column_number_density')
        .mean()
        .reduceRegion(reducer=ee.Reducer.mean(), geometry=roi)
        .getNumber('O3_column_number_density')
        .multiply(2.238)
        .getInfo()
    
    )
    
    print(f'Mean ozone column: {ozone:.3f} atm-cm')
    
    # Water vapor [cm]
    # Conversion factor from kg/m² to cm is 1/10
    water = (
    
        ee.ImageCollection('NOAA/CFSR')
        .filterDate(date, date + pd.Timedelta(1, 'd'))
        .filterBounds(roi)
        .select('Precipitable_water_entire_atmosphere_single_layer')
        .mean()
        .reduceRegion(reducer=ee.Reducer.mean(), geometry=roi)
        .getNumber('Precipitable_water_entire_atmosphere_single_layer')
        .divide(10)
        .getInfo()
    
    )
    
    print(f'Water in the atmosphere: {water:.2f} cm')
    
    # Pressure [Pa]
    # The dataset is limited to continental areas
    pressure = (
        
        ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
        .filterDate(date, date + pd.Timedelta(1, 'd'))
        .filterBounds(roi)
        .select('surface_pressure')
        .mean()
        .reduceRegion(reducer=ee.Reducer.mean(), geometry=roi)
        .getNumber('surface_pressure')
        .getInfo()
    
    )
    
    # In case no data are available, use a default value
    if pressure == None:
    
        pressure = 101320

        print(f'Atmospheric pressure: {pressure:.0f} Pa (default)')
    
    else:

        print(f'Atmospheric pressure: {pressure:.0f} Pa (calculated)')

    return aod_550, ozone, water, pressure

### Error metrics

In [2]:
def SAM(x, y):
    '''
    This functions is aimed to calculate the spectral angle mapper (SAM).

    ---
    x, y: arrays or Pandas Series contaning the data to be compared.
    '''

    # Calculate the similarity
    d = np.acos((x*y).sum() / ((x**2).sum()*(y**2).sum())**(1/2))

    return d

In [ ]:
def errorMetrics(data, ref, colnames):
    '''
    This function calculates several error metrics:
    Pearson correlation (r), mean bias error (MBE),
    mean percentage error (MPE), mean absolute error (MAE),
    mean absolute percentage error (MAPE), root mean square
    error (RMSE), root mean squared percentage error (RMSPE),
    and spectral angle mapper (SAM).

    ---
    data: a dataframe that will be evaluated (modelled).

    ref: a dataframe that will be used as reference (field).

    colnames: the name of the columns containing data to be
              assessed (must be the same for both data and ref).
    '''
    
    # Initialize variables
    r = []
    r_pvalue = []
    mbe = []
    mpe = []
    mae = []
    mape = []
    rmse = []
    rmspe = []
    sam = []
    
    # Iterate over each point
    for col in colnames:
    
        # Calculate Pearson's correlation
        correlation = stats.pearsonr(data[col], ref[col])
    
        # Append correlation results to the lists
        r.append(correlation.statistic)
        r_pvalue.append(correlation.pvalue)
    
        # Calculate error
        error = data[col] - ref[col]
    
        # Calculate mean bias error
        mbe.append(error.mean())
    
        # Calculate mean percentage error
        mpe.append(error.mean() / ref[col].mean())
    
        # Calculate mean absolute error
        mae.append(abs(error).mean())
    
        # Calculate mean absolute percentage error
        mape.append(abs(error).mean() / ref[col].mean())
    
        # Calculate root mean square error (RMSE)
        rmse.append(np.sqrt((error ** 2).mean()))
    
        # Calculate root mean square percentage error
        rmspe.append(np.sqrt((error ** 2).mean()) / ref[col].mean())

        # Calculate spectral angle mapper
        sam.append(SAM(data[col], ref[col]))
    
    # Create dataframe
    error_metrics = pd.DataFrame({'point': data.columns[:-1],
                                  'r': r, 'pvalue': r_pvalue,
                                  'mbe': mbe, 'mpe': mpe,
                                  'mae': mae, 'mape': mape,
                                  'rmse': rmse, 'rmspe': rmspe,
                                  'sam': sam})
    
    return error_metrics

### Plot field and modelled spectra

In [ ]:
def plotSpectra(data, error_metrics):
    '''
    This function is aimed at plotting the modelled and field
    spectra, as well as the main error metrics.

    ---
    data: a long dataframe containing field and modelled data.

    error_metrics: a dataframe as generated by 
                   the function error_metrics.
    '''

    # Create grid
    grid = sns.FacetGrid(data=data, col='time', col_wrap=3,
                         sharex=True, sharey=True, aspect=1.2, 
                         height=4, despine=False)
    
    # Field data
    grid.map(sns.lineplot, 'wl', 'es',
             color='#018571', label='Field')
    
    # Modelled data
    grid.map(sns.lineplot, 'wl', 'esm',
             color='#a6611a', label='Model')
    
    # Axes labels
    grid.set(xlabel=r'$\lambda ~ (nm)$',
             ylabel=r'$E_s ~ (Wm^{-2}nm^{-1})$')

    # Plot titles
    grid.set_titles('{col_name}', weight='bold')
    
    # Legend
    plt.legend(title='Source', loc='right', bbox_to_anchor=(1.8, 0.5),
               title_fontproperties={'weight': 'bold'})
    
    # Add error metrics in each plot
    for ax, r, mpe, mape, rmse, sam, in zip(grid.axes.flatten(),
                                            error_metrics.r,
                                            error_metrics.mpe,
                                            error_metrics.mape,
                                            error_metrics.rmse,
                                            error_metrics.sam):
    
        # Write metrics
        ax.annotate(f'r = {r:.2f}' +
                    f'\nMPE = {mpe:.1%}' +
                    f'\nMAPE = {mape:.1%}' +
                    f'\nRMSE = {rmse:.2f}' + r' $Wm^{-2}nm^{-1}$' +
                    f'\nSAM = {sam:.2f}',
                    xy=(0.2, 0.1), xycoords='axes fraction')

    plt.show()